![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_LongformerForQuestionAnswering.ipynb)

## Import LongformerForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 4.0.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import Longformer models trained/fine-tuned for question answering via `LongformerForQuestionAnswering` or `TFLongformerForQuestionAnswering`. These models are usually under `Question Answering` category and have `longformer` in their labels
- Reference: [TFLongformerForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/longformer#transformers.TFLongformerForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=longformer&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.39.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- DeBERTa v2&v3 use SentencePiece, so we will have to install that as well


In [1]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
pandas-

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [hf-tiny-model-private/tiny-random-LongformerForQuestionAnswering](https://huggingface.co/hf-tiny-model-private/tiny-random-LongformerForQuestionAnswering) model from HuggingFace as an example
- In addition to `TFLongformerForQuestionAnswering` we also need to save the `DebertaV2Tokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [1]:
from transformers import TFLongformerForQuestionAnswering, LongformerTokenizer
import tensorflow as tf

MODEL_NAME = 'hf-tiny-model-private/tiny-random-LongformerForQuestionAnswering'

tokenizer = LongformerTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFLongformerForQuestionAnswering.from_pretrained(MODEL_NAME)
except:
  model = TFLongformerForQuestionAnswering.from_pretrained(MODEL_NAME, from_pt=True)

@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFLongformerForQuestionAnswering.

All the layers of TFLongformerForQuestionAnswering were initialized from the model checkpoint at hf-tiny-model-private/tiny-random-LongformerForQuestionAnswering.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerForQuestionAnswering for predictions without further training.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorfl

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Initializing global attention on question tokens...


Let's have a look inside these two directories and see what we are dealing with:

In [2]:
!ls -l {MODEL_NAME}

total 556
-rw-r--r-- 1 root root    791 May 16 01:44 config.json
drwxr-xr-x 3 root root   4096 May 16 01:44 saved_model
-rw-r--r-- 1 root root 557344 May 16 01:44 tf_model.h5


In [3]:
!ls -l {MODEL_NAME}/saved_model/1

total 30808
drwxr-xr-x 2 root root     4096 May 16 01:44 assets
-rw-r--r-- 1 root root       53 May 16 01:44 fingerprint.pb
-rw-r--r-- 1 root root    89594 May 16 01:44 keras_metadata.pb
-rw-r--r-- 1 root root 31441110 May 16 01:44 saved_model.pb
drwxr-xr-x 2 root root     4096 May 16 01:44 variables


In [4]:
!ls -l {MODEL_NAME}_tokenizer

total 32
-rw-r--r-- 1 root root  4545 May 16 01:41 merges.txt
-rw-r--r-- 1 root root   964 May 16 01:41 special_tokens_map.json
-rw-r--r-- 1 root root  1218 May 16 01:41 tokenizer_config.json
-rw-r--r-- 1 root root 14630 May 16 01:41 vocab.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.json` and `merges.txt` files from the tokenizer
- All we need is to first convert vocab.json to `vocab.txt` and copy both `vocab.txt` and `merges.txt` into saved_model/1/assets which Spark NLP will look for

In [5]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in tokenizer.get_vocab().keys():
        f.write("%s\n" % item)

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}
!cp {MODEL_NAME}_tokenizer/merges.txt {asset_path}

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [6]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 16
-rw-r--r-- 1 root root 4545 May 16 01:44 merges.txt
-rw-r--r-- 1 root root 4470 May 16 01:44 vocab.txt


## Import and Save LongformerForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [7]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-05-16 01:44:29--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-05-16 01:44:30--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 an

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
#Restart here
1 + while:


In [1]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `LongformerForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `LongformerForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [3]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'hf-tiny-model-private/tiny-random-LongformerForQuestionAnswering'

spanClassifier = LongformerForQuestionAnswering.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [4]:
spanClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [5]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your LongformerForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [6]:
! ls -l {MODEL_NAME}_spark_nlp

total 31020
drwxr-xr-x 5 root root     4096 May 16 01:46 fields
-rw-r--r-- 1 root root 31755215 May 16 01:47 longformer_classification_tensorflow
drwxr-xr-x 2 root root     4096 May 16 01:46 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny LongformerForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [7]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = LongformerForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+----------------------+
|result                |
+----------------------+
|[I live in Berkeley .]|
+----------------------+



That's it! You can now go wild and use hundreds of `LongformerForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
